In [ ]:
!nvidia-smi

In [ ]:
# !pip install keras==2.2.4
# !pip install tensorflow-gpu==1.12.0

In [1]:
import tensorflow as tf
import keras
print(tf.__version__,keras.__version__)
import os,sys,shap
sys.path.append('/home/ubuntu/hacking/projects/deep-mediation/deep-mediation/src')
import utils,build_model
import innvestigate,shap
import numpy as np
from nilearn.image import load_img,mean_img
from keras.models import load_model
import nibabel as nib
import create_dataset,utils
import pandas as pd
from nilearn.image import resample_img,concat_imgs,load_img,resample_to_img,threshold_img,smooth_img
print(shap.__version__)

/home/ubuntu/anaconda3/envs/myclone/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/myclone/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/myclone/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/myclone/lib/pyt

1.12.0 2.2.4


Python 3.5 support is deprecated and will be removed in a future release. Consider switching to Python 3.6 or 3.7


0.35.0


### Specify the data path and the model to use for visualization purpose

In [2]:
data_path = '/home/ubuntu/hacking/data/stephan-data-ni-files'
model_to_use = '/home/ubuntu/hacking/projects/deep-mediation/deep-mediation/results/deep/deep-iter-4-run-0.h5'
img_to_use = '/home/ubuntu/hacking/data/stephan-data-ni-files/BMRK5_data/stim_bmrk5_S860_OC1339_zs.nii'

In [3]:
num_imgs = 2
dataset = create_dataset.generate_dataset(data_path,test_data_size=0.30)
test_rate,test_temp,test_imgs_list,flat_test_rate,flat_test_rate_zs = utils.get_rate_temp_img(dataset,subjs='test_subjs')
train_rate,train_temp,train_imgs_list,flat_train_rate,flat_train_rate_zs = utils.get_rate_temp_img(dataset,subjs='train_subjs')

print("Using %s random images from training dataset as a background signal for Deep Shap..."%num_imgs)
random_train_imgs_list = np.array(train_imgs_list)[np.random.randint(0,146,num_imgs)]
random_train_imgs = utils.concat_images(list(random_train_imgs_list),None)
print(random_train_imgs.shape)


# background = utils.concat_images(list(train_imgs_list),None)
# background = np.rollaxis(background.get_fdata(), 3, 0)[...,None]

background = np.rollaxis(random_train_imgs.get_fdata(), 3, 0)[...,None]
test_data = utils.concat_images(list(test_imgs_list),None)
test_data = np.rollaxis(test_data.get_fdata(), 3, 0)[...,None]

These are the studies to use ['NSF', 'BMRK3', 'BMRK4', 'IE', 'ILCP', 'EXP', 'SCEBL']
This is the test study BMRK5 
Using 2 random images from training dataset as a background signal for Deep Shap...
(91, 109, 91, 82)


In [4]:
print(background.shape,test_data.shape)

(82, 91, 109, 91, 1) (2596, 91, 109, 91, 1)


In [5]:
model = load_model(model_to_use)

In [6]:
def get_shap_values(model, background,test_data):
    explainer = shap.DeepExplainer(model, background,keras.backend.get_session())
#     explainer = shap.GradientExplainer(model, background,keras.backend.get_session())
    values = []
    for i in range(0,1000):
        print(i)
#         shap_values = explainer.shap_values(test_data[i:i+1,:],check_additivity=False)
        shap_values = explainer.shap_values(test_data[i:i+1,:],check_additivity=False)
        values.append(shap_values[0])
    values_ = np.concatenate(values)
    return values_

In [ ]:
values_ = get_shap_values(model, background,test_data)

keras is no longer supported, please use tf.keras instead.


0


In [ ]:
model = load_model('/home/ubuntu/hacking/projects/deep-mediation/deep-mediation/results/deep/deep-iter-4-run-0.h5')

In [ ]:
dataset = create_dataset.generate_dataset(data_path,test_data_size=0.30)
train_rate,train_temp,train_imgs_list,flat_train_rate,flat_train_rate_zs = utils.get_rate_temp_img(dataset,subjs='train_subjs')
# val_rate,val_temp,val_imgs_list,flat_val_rate,flat_val_rate_zs = utils.get_rate_temp_img(dataset,subjs='val_subjs')
test_rate,test_temp,test_imgs_list,flat_test_rate,flat_test_rate_zs = utils.get_rate_temp_img(dataset,subjs='test_subjs')


In [ ]:
# dataset = create_dataset.generate_dataset(data_path,test_data_size=0.30)
# test_rate,test_temp,test_imgs_list,flat_test_rate,flat_test_rate_zs = utils.get_rate_temp_img(dataset,subjs='test_subjs')
# df_test = pd.DataFrame()
# df_test['X'] = test_temp
# df_test['Y'] = flat_test_rate_zs

In [ ]:
train_imgs_list[0:5],test_imgs_list[0:5]

In [ ]:
print("Reading training images...")
train_imgs = utils.concat_imgs(train_imgs_list)
print("Smoothing training images...")
train_imgs = smooth_img(train_imgs, fwhm=[4,4,4]) #in mm 
train_imgs = np.rollaxis(train_imgs.get_fdata(), 3, 0)[...,None]

print("Reading testing images...")
test_imgs = utils.concat_imgs(test_imgs_list)
print("Smoothing testing images...")
test_imgs = smooth_img(test_imgs, fwhm=[4,4,4]) #in mm 
test_imgs = np.rollaxis(test_imgs.get_fdata(), 3, 0)[...,None]

In [ ]:
def get_model(model_to_use):
    model_with_gpu = load_model(model_to_use)
    model = model_with_gpu.layers[-2]
    return model
def get_nifti_params(input_img):
    affine = input_img.affine
    hdr = input_img.header
    return affine,hdr
def get_nifti_image(array,affine,header):
    nii_image = nib.Nifti1Image(array, affine, header)
    return nii_image

In [ ]:
input_img = load_img(img_to_use)
affine,hdr = get_nifti_params(input_img)
input_img = input_img.get_fdata()
input_img = np.rollaxis(input_img, 3,0)[...,None]
# print(input_img.shape)

In [ ]:
model= get_model(model_to_use)
model.summary()

In [ ]:
test_imgs.shape

In [ ]:
num_imgs = 2
dataset = create_dataset.generate_dataset(data_path,test_data_size=0.30)
test_rate,test_temp,test_imgs_list,flat_test_rate,flat_test_rate_zs = utils.get_rate_temp_img(dataset,subjs='test_subjs')
train_rate,train_temp,train_imgs_list,flat_train_rate,flat_train_rate_zs = utils.get_rate_temp_img(dataset,subjs='train_subjs')

print("Using %s random images from training dataset as a background signal for Deep Shap..."%num_imgs)
random_train_imgs_list = np.array(train_imgs_list)[np.random.randint(0,146,num_imgs)]
random_train_imgs = utils.concat_images(list(random_train_imgs_list),None)
print(random_train_imgs.shape)


# background = utils.concat_images(list(train_imgs_list),None)
# background = np.rollaxis(background.get_fdata(), 3, 0)[...,None]

background = np.rollaxis(random_train_imgs.get_fdata(), 3, 0)[...,None]
test_data = utils.concat_images(list(test_imgs_list[0:10]),None)
test_data = np.rollaxis(test_data.get_fdata(), 3, 0)[...,None]

In [ ]:
test_data = utils.concat_images(list(test_imgs_list[0:10]),None)
test_data = np.rollaxis(test_data.get_fdata(), 3, 0)[...,None]

In [ ]:
background = background[0:10,:]
background.shape

In [ ]:
def get_shap_values(model, background,test_data):
#     explainer = shap.DeepExplainer(model, background)
    explainer = shap.GradientExplainer(model, background)
    values = []
    for i in range(0,test_data.shape[0]):
        print(i)
        shap_values = explainer.shap_values(test_data[i:i+1,:],check_additivity=False)
        values.append(shap_values[0])
    values_ = np.concatenate(values)
    return values_

In [ ]:
test_data.shape

In [ ]:
values_ = get_shap_values(model, background,test_data)

In [ ]:
explainer = shap.GradientExplainer(model.layers[-2], input_img,batch_size=1)

In [ ]:
values = []
for i in range(0,input_img.shape[0]):
    print(i)
    shap_values = explainer.shap_values(input_img[i:i+1,:])
    values.append(shap_values[0])
values_ = np.concatenate(values)

In [ ]:
e = shap.GradientExplainer(model.layers[-2], input_img)

In [ ]:
shap_values = e.shap_values(input_img, nsamples=100)